<a href="https://colab.research.google.com/github/NID123-CH/LLM-Codes/blob/main/Copy_of_Generating_Images_in_Batches_with_Stable_Diffusion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Setup input + output

In [ ]:
!nvidia-smi

Thu Dec 21 16:16:16 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-SXM2-16GB           Off | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P0              27W / 300W |      0MiB / 16384MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
#set params & select model

model = "stabilityai/stable-diffusion-xl-base-1.0"

In [ ]:
#set up folder configuration
import os
from google.colab import files, drive

drive.mount('/content/drive')

PROJECT_NAME = 'artstyle3'

BASE_FOLDER = os.path.join(os.getcwd(), 'drive', 'MyDrive','Colab Notebooks')
OUTPUT_IMAGE_FOLDER = os.path.join(BASE_FOLDER, 'Projects', PROJECT_NAME)

if not os.path.exists(OUTPUT_IMAGE_FOLDER):
    print(f'{OUTPUT_IMAGE_FOLDER} was not present and was created')
    os.makedirs(OUTPUT_IMAGE_FOLDER, exist_ok=True)

Mounted at /content/drive


# Load SDXL pipeline


In [ ]:
!pip install transformers accelerate scipy safetensors diffusers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 18.7 MB/s eta 0:00:00


In [ ]:
#import libraries
from diffusers import StableDiffusionXLPipeline #to use the refiner import DiffusionPipeline, StableDiffusionXLImg2ImgPipeline
import torch

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [ ]:
#Load SDXL pipeline
pipe = StableDiffusionXLPipeline.from_pretrained(
  model,
  torch_dtype=torch.float16,
  use_safetensors=True,
  variant="fp16"
)
pipe.to("cuda")

print(f'{model} loaded')

model_index.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

Fetching 19 files:   0%|          | 0/19 [00:00<?, ?it/s]

scheduler/scheduler_config.json:   0%|          | 0.00/479 [00:00<?, ?B/s]

text_encoder/config.json:   0%|          | 0.00/565 [00:00<?, ?B/s]

tokenizer/merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

...

diffusion_pytorch_model.fp16.safetensors:   0%|          | 0.00/5.14G [00:00<?, ?B/s]

diffusion_pytorch_model.fp16.safetensors:   0%|          | 0.00/167M [00:00<?, ?B/s]

diffusion_pytorch_model.fp16.safetensors:   0%|          | 0.00/167M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

stabilityai/stable-diffusion-xl-base-1.0 loaded


#Generate images

In [ ]:
#gets the index of the last generated image
def get_last_index(imgfolder): #os.listdir(OUTPUT_IMAGE_FOLDER)
  outfiles = [file for file in os.listdir(imgfolder) if os.path.isfile(os.path.join(imgfolder,file))]
  if len(outfiles)>0:
    try:
      img_num = sorted([int(img.split('.')[0]) for img in os.listdir(imgfolder) if img.split('.')[0].isdigit()])[-1] #gets last image in the output folder (highest number)
      return img_num
    except:
      return 0
  else:
    return 0

get_last_index(OUTPUT_IMAGE_FOLDER)

0

In [ ]:
import random
import pandas as pd

art_styles = ['Rembrandt',  'Magritte', 'Monet', 'Klimt', 'Renoir', 'Peter Bruegel', 'Chagall',
              'Hokusai', 'Hiroshige','Steampunk', 'Art Deco', 'Art Nouveau', 'Paul Gauguin',
              'Banksy', 'Goya', 'Van Gogh', 'Leonardo da Vinci']
art_types = ['beautiful lansdcape', 'medieval town', 'sunset view from a park', 'desert',
             'modern city at night, Berlin', 'Potsdamer Platz', 'arctic landscape with aurora borealis',
             'portrait of mother and child', 'musical instruments', 'jazz band playing music', 'guitar player',
             'skyscrappers', 'erupting volcano', 'sunny beach and sea', 'portrait of a woman', 'view of Berlin',
             'spaeti shop in Berlin', 'AlexanderPlatz', 'cafe at night',
             'desert', 'jungle', 'painting of mechanical device', 'supercomputer','river bank', 'view of a city with bridge',
             'still nature', 'wolf', 'a view of the Spree', 'crows in Berlin', 'crows and foxes in Berlin Tiergarten']

img_num = get_last_index(OUTPUT_IMAGE_FOLDER)

STARTING_RANGE = img_num+1 if img_num!=0 else 0 #starts image index after last index or 0 if no images found

ADD_RANGE = 200 #how many images to generate

prompt_save = [] #this keeps the info for the experiment sheet

for i, seed in enumerate(range(STARTING_RANGE, STARTING_RANGE+ADD_RANGE)):
    generator = torch.Generator("cuda").manual_seed(seed)
    art_style = random.choice(art_styles)
    art_type = random.choice(art_types)
    prompt = f'{art_type} in the style of {art_style}'
    print(f'Processing image {i+1} / {ADD_RANGE} | seed: {seed} | prompt: {prompt}')
    negative_prompt = None
    image = pipe(prompt=prompt,
                negative_prompt=negative_prompt,
                generator=generator,
                num_inference_steps=25)
    image = image.images[0]
    savefilename = f'{seed}_{prompt}.png'
    image.save(os.path.join(OUTPUT_IMAGE_FOLDER, savefilename))
    prompt_save.append([savefilename, seed, art_style, prompt, negative_prompt])

Processing image 1 / 200 | seed: 0 | prompt: sunset view from a park in the style of Banksy


  0%|          | 0/25 [00:00<?, ?it/s]

Processing image 2 / 200 | seed: 1 | prompt: crows and foxes in Berlin Tiergarten in the style of Paul Gauguin


  0%|          | 0/25 [00:00<?, ?it/s]

Processing image 3 / 200 | seed: 2 | prompt: portrait of a woman in the style of Art Nouveau


...

  0%|          | 0/25 [00:00<?, ?it/s]

Processing image 199 / 200 | seed: 198 | prompt: jazz band playing music in the style of Rembrandt


  0%|          | 0/25 [00:00<?, ?it/s]

Processing image 200 / 200 | seed: 199 | prompt: crows and foxes in Berlin Tiergarten in the style of Van Gogh


  0%|          | 0/25 [00:00<?, ?it/s]

In [ ]:
prompt_file = pd.DataFrame(prompt_save, columns=['filename', 'seed','art_style', 'prompt', 'negative_prompt'])
prompt_file.to_csv(os.path.join(OUTPUT_IMAGE_FOLDER, f'trial_up_to_{seed}.csv'), index=False)
prompt_file

,filename,seed,art_style,prompt,negative_prompt
0,0_sunset view from a park in the style of Bank...,0,Banksy,sunset view from a park in the style of Banksy,None
1,1_crows and foxes in Berlin Tiergarten in the ...,1,Paul Gauguin,crows and foxes in Berlin Tiergarten in the st...,None
2,2_portrait of a woman in the style of Art Nouv...,2,Art Nouveau,portrait of a woman in the style of Art Nouveau,None
3,3_medieval town in the style of Peter Bruegel.png,3,Peter Bruegel,medieval town in the style of Peter Bruegel,None
4,4_portrait of a woman in the style of Klimt.png,4,Klimt,portrait of a woman in the style of Klimt,None
...,...,...,...,...,...
195,195_view of a city with bridge in the style of...,195,Peter Bruegel,view of a city with bridge in the style of Pet...,None
196,196_sunny beach and sea in the style of Goya.png,196,Goya,sunny beach and sea in the style of Goya,None
197,197_desert in the style of Klimt.png,197,Klimt,desert in the style of Klimt,None
198,198_jazz band playing music in the style of Re...,198,Rembrandt,jazz band playing music in the style of Rembrandt,None
